In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import nltk
import re
import numpy as np 
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
from wordcloud import WordCloud, STOPWORDS

df = pd.read_csv(r"C:\Users\ymf\Desktop\pic16bGit\rye_data_project_spring2021\Reddit_dataset\Adulting2016-2021.csv")
df

,Unnamed: 0,author,created_utc,domain,full_link,num_comments,over_18,score,selftext,subreddit,title,num_crossposts
0,0,paradisenoir,1469810090,self.Adulting,https://www.reddit.com/r/Adulting/comments/4v7...,1,False,1,I seriously didn't know where to put this ques...,Adulting,Need help with Updating driver's License,NaN
1,1,emptysometimes,1469550546,youtu.be,https://www.reddit.com/r/Adulting/comments/4up...,0,False,1,NaN,Adulting,Adulting 101,NaN
2,2,PortlandPerson94,1468814980,self.Adulting,https://www.reddit.com/r/Adulting/comments/4td...,1,False,4,"How to do laundry, organize your filing cabine...",Adulting,"in the spirit of the discription of this sub, ...",NaN
3,3,thisisthetyty,1467816249,youtube.com,https://www.reddit.com/r/Adulting/comments/4ri...,0,False,1,NaN,Adulting,New youtube video about symptoms of adulting. ...,NaN
4,4,writer318,1467729176,theodysseyonline.com,https://www.reddit.com/r/Adulting/comments/4rc...,0,False,1,NaN,Adulting,"Why Must We Have To ""Adult""?",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9397,9397,PortlandPerson94,1468814980,self.Adulting,https://www.reddit.com/r/Adulting/comments/4td...,1,False,4,"How to do laundry, organize your filing cabine...",Adulting,"in the spirit of the discription of this sub, ...",NaN
9398,9398,thisisthetyty,1467816249,youtube.com,https://www.reddit.com/r/Adulting/comments/4ri...,0,False,1,NaN,Adulting,New youtube video about symptoms of adulting. ...,NaN
9399,9399,writer318,1467729176,theodysseyonline.com,https://www.reddit.com/r/Adulting/comments/4rc...,0,False,1,NaN,Adulting,"Why Must We Have To ""Adult""?",NaN
9400,9400,apolloxijedi,1467437994,self.Adulting,https://www.reddit.com/r/Adulting/comments/4qw...,3,False,4,So the short version of the story my fiance an...,Adulting,can't believe I'm 30,NaN


In [3]:
#Take interesting columns
df1=df[['selftext','num_comments','score','title']]
#drop those with text=NA
df1=df1.dropna(subset=['selftext'])
#drop meaningless text
df1=df1[(df1.selftext!='[removed]') & (df1.selftext!='[deleted]')].reset_index()
#drop duplicate
duplicate = df1.duplicated()
print('There are', duplicate.sum(), 'duplicated values.')
df1=df1.drop_duplicates()
len(df1)

There are 0 duplicated values.


6518

In [4]:
#convert to lower case
df1['selftext']=df1['selftext'].str.lower()
df1['selftext']

0       i seriously didn't know where to put this ques...
1       how to do laundry, organize your filing cabine...
2       so the short version of the story my fiance an...
3       i am tired of myself and other adults not be o...
4       finally mastering public transportation. i fee...
                              ...                        
6513    like which info do i give them? my policy numb...
6514    i seriously didn't know where to put this ques...
6515    how to do laundry, organize your filing cabine...
6516    so the short version of the story my fiance an...
6517    i am tired of myself and other adults not be o...
Name: selftext, Length: 6518, dtype: object

In [5]:
#remove emoji
def remove_emoji(text):
    text = text.encode('ascii', 'ignore').decode()
    return text

In [6]:
df1['selftext']=df1['selftext'].apply(lambda x: remove_emoji(x))
df1['selftext'][0]

"i seriously didn't know where to put this question, i hope this is an okay place. \n\na little background: so i'm 22 with a valid driver's license from my home state that still has my parents address on it. i actually moved to the state/town i live in several years ago, but have been a bit of a wanderer and rarely had a permanent address so i just didn't want to deal with the hassle of updating my driver's license.\n\n now it's time to do it and i think i may have trouble proving that i'm a resident of this state, even though i've been here for a few years. currently am living with a friend in the most permanent situation i've been in and probably will be for a while longer. he is the one on the lease and has bills in his name, and we only have like a verbal agreement to pay rent/bills, no contract. i don't have any bills of my own or official mail. my w-2 this year had an old address on it that i'm not associated at all with anymore. i have a pay stub, and could get my friend to writ

In [7]:
#now we remove url, punctuations, special characters
import re
def remove_other(x):
    x = re.sub("\$"," ", x) #remove $
    x = re.sub("https*\S+", " ", x) #remove url
    x = re.sub("\'\w+", '', x) #remove i'm,we're,let's after the '
    x = re.sub("[0-9]+", '', x) #remove numbers
    return x

In [8]:
df1['selftext']=df1['selftext'].apply(lambda x: remove_other(x))
df1

,index,selftext,num_comments,score,title
0,0,i seriously didn know where to put this questi...,1,1,Need help with Updating driver's License
1,2,"how to do laundry, organize your filing cabine...",1,4,"in the spirit of the discription of this sub, ..."
2,5,so the short version of the story my fiance an...,3,4,can't believe I'm 30
3,6,i am tired of myself and other adults not be o...,1,5,Can't we just be open and honest
4,8,finally mastering public transportation. i fee...,0,2,Public transportation
...,...,...,...,...,...
6513,9394,like which info do i give them? my policy numb...,5,1,"When you get into a car crash, what exactly do..."
6514,9395,i seriously didn know where to put this questi...,1,1,Need help with Updating driver's License
6515,9397,"how to do laundry, organize your filing cabine...",1,4,"in the spirit of the discription of this sub, ..."
6516,9400,so the short version of the story my fiance an...,3,4,can't believe I'm 30


Now we use TextBlob for sentiment analysis 

In [9]:
from textblob import TextBlob

In [11]:
pol = []
sub = []

for text in df1['selftext']:
    selftext = TextBlob(text)
    pol.append(selftext.sentiment.polarity)
    sub.append(selftext.sentiment.subjectivity)
    
    
# assign data of lists.  
data = {'Title':df1['title'],'Text': df1['selftext'], 'polarity': pol, 'subjectivity':sub}  
  
# Create DataFrame  
df2 = pd.DataFrame(data)  
df2

,Title,Text,polarity,subjectivity
0,Need help with Updating driver's License,i seriously didn know where to put this questi...,0.074443,0.427451
1,"in the spirit of the discription of this sub, ...","how to do laundry, organize your filing cabine...",0.600000,1.000000
2,can't believe I'm 30,so the short version of the story my fiance an...,-0.040000,0.540000
3,Can't we just be open and honest,i am tired of myself and other adults not be o...,0.025000,0.508333
4,Public transportation,finally mastering public transportation. i fee...,0.000000,0.533333
...,...,...,...,...
6513,"When you get into a car crash, what exactly do...",like which info do i give them? my policy numb...,0.000000,0.000000
6514,Need help with Updating driver's License,i seriously didn know where to put this questi...,0.074443,0.427451
6515,"in the spirit of the discription of this sub, ...","how to do laundry, organize your filing cabine...",0.600000,1.000000
6516,can't believe I'm 30,so the short version of the story my fiance an...,-0.040000,0.540000


In [12]:
df2['pol_estimate']=0
df2['pol_estimate'].loc[df2['polarity']>0.2]=1
df2['pol_estimate'].loc[df2['polarity']< -0.2]=-1
df2['sub_estimate']=0
df2['sub_estimate'].loc[df2['subjectivity']>0.6]=1
df2['sub_estimate'].loc[df2['subjectivity']<0.4]=-1
df2

C:\Users\ymf\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\ymf\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\ymf\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc.

,Title,Text,polarity,subjectivity,pol_estimate,sub_estimate
0,Need help with Updating driver's License,i seriously didn know where to put this questi...,0.074443,0.427451,0,0
1,"in the spirit of the discription of this sub, ...","how to do laundry, organize your filing cabine...",0.600000,1.000000,1,1
2,can't believe I'm 30,so the short version of the story my fiance an...,-0.040000,0.540000,0,0
3,Can't we just be open and honest,i am tired of myself and other adults not be o...,0.025000,0.508333,0,0
4,Public transportation,finally mastering public transportation. i fee...,0.000000,0.533333,0,0
...,...,...,...,...,...,...
6513,"When you get into a car crash, what exactly do...",like which info do i give them? my policy numb...,0.000000,0.000000,0,-1
6514,Need help with Updating driver's License,i seriously didn know where to put this questi...,0.074443,0.427451,0,0
6515,"in the spirit of the discription of this sub, ...","how to do laundry, organize your filing cabine...",0.600000,1.000000,1,1
6516,can't believe I'm 30,so the short version of the story my fiance an...,-0.040000,0.540000,0,0


In [13]:
pol_mean = df2['polarity'].mean()
pol_mean

0.0887435311604034

In [14]:
sub_mean = df2['subjectivity'].mean()
sub_mean

0.46625098458473985

In [15]:
overall_pol = pol_mean = df2['pol_estimate'].mean()
overall_pol

0.15403498005523167

In [16]:
overall_sub = pol_mean = df2['sub_estimate'].mean()
overall_sub

-0.10601411475912857

In [17]:
index=df2[df2['Text'].str.contains("great")].index
#example of calculating average sentiment score of a given word in any posts
df2.iloc[index]['polarity'].mean()

0.1375310117580357

In [18]:
index=df2[df2['Text'].str.contains("great")].index
#example of calculating average sentiment score of a given word in positive posts
df2.iloc[index][df2['pol_estimate']==1]['polarity'].mean()

<ipython-input-18-e7ca6f9e5f30>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2.iloc[index][df2['pol_estimate']==1]['polarity'].mean()


0.2728283116550691

In [19]:
index=df2[df2['Text'].str.contains("bad")].index
#example of calculating average sentiment score of a given word in negative posts
df2.iloc[index][df2['pol_estimate']==-1]['polarity'].mean()

<ipython-input-19-6f3d9e536141>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2.iloc[index][df2['pol_estimate']==-1]['polarity'].mean()


-0.37340054304550097